In [ ]:
import polars as pl

# Step 1: Load the transactions data
# This contains all transaction details
df_transactions = pl.read_csv("../data/transactions_data.csv")

# Step 2: Load the train_fraud_labels data
# This contains transaction_ids we want to keep
df_fraud_labels = pl.read_csv("../data/train_fraud_labels.csv")

# Step 3: Filter transactions_data to keep only those in train_fraud_labels
# Join on 'id' from transactions and 'transaction_id' from fraud labels, keeping only matches 13m -8.9m
df_filtered_transactions = df_transactions.join(
    df_fraud_labels,
    left_on="id",
    right_on="transaction_id",
    how="inner"  # Only keeps transactions present in both datasets
)

# Step 4: Load the cards data to get card_brand information
df_cards = pl.read_csv("../data/cards_data.csv")

# Step 5: Join the filtered transactions with cards data to get card_brand
# Join on 'card_id' from transactions and 'id' from cards
df_combined = df_filtered_transactions.join(
    df_cards,
    left_on="card_id",
    right_on="id",
    how="left"  # Left join to keep all transactions. If a card_id doesn't match, it will still be included with null card_brand
)

# Step 6: Filter for transactions where card_brand is "Amex" 8.9m 
df_amex_transactions = df_combined.filter(pl.col("card_brand") == "Amex")

# Step 7: Select exactly the specified columns for the output
output_df = df_amex_transactions.select([
    "id",              # Transaction ID
    "date",           # Transaction date
    "client_id",      # Client ID
    "card_id",        # Card ID
    "amount",         # Transaction amount
    "merchant_id",    # Merchant ID
    "card_brand",     # Card brand (will all be Amex)
    "fraud_label",   # Fraud label from train_fraud_labels
    "mcc"         # Merchant category code
])

# Step 8: Save the filtered Amex transactions to a new CSV file
output_df.write_csv("../data/amex_filtered_transactions.csv")

# Step 9: Print confirmation and row count
print(f"Saved {output_df.height} Amex transactions to 'amex_filtered_transactions.csv'")

Saved 572424 Amex transactions to 'amex_filtered_transactions.csv'


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_date, when
from pyspark.sql.types import FloatType

# Initialize Spark session - This starts our PySpark environment
spark = SparkSession.builder.appName("DataPreprocessing").getOrCreate()

# Define file locations in cloud storage
file_location_cards = "/FileStore/tables/cards_data.csv"
file_location_users = "/FileStore/tables/users_data.csv"
file_location_amex = "/FileStore/tables/amex_filtered_transactions.csv"

# Step 1: Load the datasets from cloud storage
# Cards data contains card details like credit limit and PIN change info
df_cards = spark.read.csv(file_location_cards, header=True, inferSchema=True)
# Users data has user info like income and age
df_users = spark.read.csv(file_location_users, header=True, inferSchema=True)
# Amex filtered transactions from our previous work
df_amex_transactions = spark.read.csv(file_location_amex, header=True, inferSchema=True)

# Step 2: Preprocess cards data (kept as DataFrame)
# Drop the 'card_on_dark_web' column since we don’t need it
df_cards = df_cards.drop("card_on_dark_web")
# Clean 'credit_limit' by removing '$' and converting to float
df_cards = df_cards.withColumn(
    "credit_limit",
    regexp_replace(col("credit_limit"), "[\$,]", "").cast(FloatType())
)
# Convert 'acct_open_date' to a proper date format (MM/YYYY)
df_cards = df_cards.withColumn(
    "acct_open_date",
    to_date(col("acct_open_date"), "MM/yyyy")
)
# Flag if a PIN change is due - if last changed over 7 years ago
current_year = 2025  # Using 2025 based on your context
df_cards = df_cards.withColumn(
    "PIN_Change_Due",
    when(col("year_pin_last_changed") < current_year - 7, "Yes").otherwise("No")
)

# Step 3: Preprocess users data (kept as DataFrame)
# Clean financial columns by removing '$' and converting to float
df_users = df_users.withColumn(
    "per_capita_income",
    regexp_replace(col("per_capita_income"), "[\$,]", "").cast(FloatType())
).withColumn(
    "yearly_income",
    regexp_replace(col("yearly_income"), "[\$,]", "").cast(FloatType())
).withColumn(
    "total_debt",
    regexp_replace(col("total_debt"), "[\$,]", "").cast(FloatType())
)
# Add 'retirement_status' - Retired if current age meets or exceeds retirement age
df_users = df_users.withColumn(
    "retirement_status",
    when(col("current_age") >= col("retirement_age"), "Retired").otherwise("Not Retired")
)
# Categorize 'age_group' based on current age
df_users = df_users.withColumn(
    "age_group",
    when(col("current_age") <= 30, "18-30")
    .when(col("current_age") <= 45, "31-45")
    .when(col("current_age") <= 60, "46-60")
    .otherwise("60+")
)
# Calculate 'Debt_to_Income_Ratio' as total debt divided by yearly income
df_users = df_users.withColumn(
    "Debt_to_Income_Ratio",
    col("total_debt") / col("yearly_income")
)

# Step 4: Preprocess amex_filtered_transactions (kept as DataFrame)
# Clean 'amount' by removing '$' and converting to float
df_amex_transactions = df_amex_transactions.withColumn(
    "amount",
    regexp_replace(col("amount"), "[\$,]", "").cast(FloatType())
)

# Step 5: Show a preview of the preprocessed DataFrames
# Let’s peek at the first few rows to confirm our changes
print("Preview of preprocessed cards data:")
df_cards.show(5, truncate=False)
print("Preview of preprocessed users data:")
df_users.show(5, truncate=False)
print("Preview of preprocessed Amex transactions (with cleaned amount):")
df_amex_transactions.show(5, truncate=False)

# Step 6: Count rows and confirm we’re done
# We’ll count the rows in each DataFrame to see what we’re working with
cards_count = df_cards.count()
users_count = df_users.count()
amex_count = df_amex_transactions.count()
print(f"Cards DataFrame has {cards_count} rows")
print(f"Users DataFrame has {users_count} rows")
print(f"Amex Transactions DataFrame has {amex_count} rows")

# Note: Spark session remains active since we’re keeping DataFrames in memory
# Use spark.stop() later when you’re done with these DataFrames